# 필요한 라이브러리 Import

In [2]:
import seaborn as sns
import matplotlib.font_manager as fm

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import warnings
%matplotlib inline

warnings.filterwarnings(action='ignore')

pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 50)

import os
import matplotlib as mpl

## 데이터 로드

In [2]:
df = pd.read_csv('./dataset/sfites_real_mix_data_full.csv')

## 필요한 데이터셋 스플릿

In [3]:
# "1_JRCODE" 컬럼의 마지막 문자가 "C1"인 행만 필터링
c1_rows = df['RUBBER'].str.endswith('C1', na=False)
c1_filtered_df = df[c1_rows]

# f!만
non_c1_filtered_df = df[~c1_rows]

# f1중에서 c1필요없는것
c1_rows_2 = non_c1_filtered_df['1_JRCODE'].str.endswith('C1', na=False)
non_c1_f1 = non_c1_filtered_df[~c1_rows_2]

# f1중에서 c1필요한 것
c1_filtered_df_2 = non_c1_filtered_df[c1_rows_2]

## !전처리 시작

### 1_JRCODE가 C1인 행에 대해 필터링하고 그룹내 개수가 10개 이상인것만 다시 필터링하여 이상치 제거

In [4]:
%%time

# 러버 그룹별로 그룹내의 요소 개수가 10이상인 것을 찾고, iqr로 이상치 제거

# "1_JRCODE" 컬럼의 마지막 문자가 "C1"인 행만 필터링
c1_rows = df['1_JRCODE'].str.endswith('C1', na=False)
c1_filtered_df = df[c1_rows]

# 원본 데이터프레임(df)에서 "RUBBER" 값이 c1_filtered_df의 "" 값과 일치하는 행만 선택
matching_rows = df[df['RUBBER'].isin(c1_filtered_df['1_JRCODE'])]

# "RUBBER" 컬럼의 값이 같은 행만 필터링
rubber_filtered_df = matching_rows[matching_rows.duplicated('RUBBER', keep=False)]

# 10개 이상의 데이터를 가진 그룹만 필터링
group_counts = matching_rows['RUBBER'].value_counts()
valid_groups = group_counts[group_counts >= 10].index
valid_df = matching_rows[matching_rows['RUBBER'].isin(valid_groups)]

selected_cols = valid_df.filter(regex='RPM[1-9]|RPM10|TEMP[1-9]|TEMP[1-2][0-9]|TIME[1-9]|TIME[1-2][0-9]|JUK[1-9]|JUK[1-2][0-9]|[1-9]_PHR|[1-2][0-9]_PHR')
len(selected_cols.columns)

# 숫자형 변수만 선택
#numeric_cols = valid_df.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = selected_cols.columns

# IQR을 이용한 이상치 제거 함수
def remove_outliers(group):
    for col in numeric_cols:
#         print(group[col].isna().sum(), len(group))
        if group[col].isna().sum() == len(group):
            continue
        else:
            Q1 = group[col].quantile(0.25)
            Q3 = group[col].quantile(0.75)
            IQR = Q3 - Q1
            group = group[(group[col] >= (Q1 - 3 * IQR)) & (group[col] <= (Q3 + 3 * IQR))]
    return group

# 그룹별로 이상치 제거 적용
filtered_df = valid_df.groupby('RUBBER').apply(remove_outliers).reset_index(drop=True)

CPU times: total: 48.4 s
Wall time: 48.7 s


### 이상치가 제거된 C1그룹들에 대한 공정조건들 평균값 계산 (재료코드 제외)

In [5]:
# 'RUBBER' 컬럼을 기준으로 각 컬럼의 평균값 계산
grouped_df = filtered_df.groupby('RUBBER').mean(numeric_only=True)
grouped_df.reset_index(inplace=True)

### 평균값이 계산된 C1집합과 F1에서 유니크한 키를 사용해서 교집합 데이터를 추출

In [6]:
matching_rows_df = c1_filtered_df_2[c1_filtered_df_2['1_JRCODE'].isin(grouped_df['RUBBER'])]

### 필요에 맞게 여러가지 컬럼 집합 리스트를 생성

In [7]:
# 병합하고자 하는 컬럼 리스트 생성

pro_cols = [f'RPM{i}' for i in np.arange(1,21)] + [f'TEMP{i}' for i in np.arange(1,21)] + [f'TIME{i}' for i in np.arange(1,21)] + [f'JUK{i}' for i in np.arange(1,21)]
#jrcode_cols = [f'{i}_JRCODE' for i in np.arange(1,22)] + [f'{i}_PHR' for i in np.arange(1,22)] + [f'{i}_PUTGB' for i in np.arange(1,22)]
jrcode_cols = [f'{i}_PHR' for i in np.arange(1,22)] + [f'{i}_PUTGB' for i in np.arange(1,22)]

exce = [f'{i}_JRCODE' for i in np.arange(1,22)] + list(matching_rows_df[['LOTNO','RUBBER','MECHCD','HS','HS_RESULT','SG','SG_RESULT','TS','TS_RESULT','EB','EB_RESULT','MNY','MNY_RESULT','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','REHO_RESULT','SCR','SCR_RESULT','HS_TIME','HS_TEMP','SG_TIME','SG_TEMP','TS_TIME','TS_TEMP','EB_TIME','EB_TEMP','MNY_TIME','MNY_TEMP','REHO_TIME','REHO_TEMP','SCR_TIME','SCR_TEMP']].columns)

### F1 RUBBER 컬럼 그룹별로 하나의 행만 추출 (재료 코드 추출)

In [8]:
# RUBBER를 그룹별로 묶고, 그룹별로 첫 번째 행만 추출하되, 'JRCODE'가 들어간 컬럼과 'RUBBER' 컬럼만 포함
grouped_filtered_df = filtered_df.groupby('RUBBER').apply(
    lambda x: x[['RUBBER'] + [col for col in x.columns if 'JRCODE' in col]].head(1)
).reset_index(drop=True)

### C1그룹에서 평균값을 계산한 공정조건들과 재료코드를 F1 데이터와 Merge

In [9]:
# 접미사 '_c'를 붙여서 병합
merged_df = pd.merge(matching_rows_df, grouped_df.add_suffix('_c'), how='inner', left_on='1_JRCODE', right_on='RUBBER_c')

# 병합 후 필요한 컬럼만 선택
final_cols_from_matching = [col for col in exce if col in merged_df.columns]  # exce에서 가져올 컬럼
final_cols_from_grouped = [f"{col}_c" for col in pro_cols + jrcode_cols if f"{col}_c" in merged_df.columns]  # grouped_df에서 가져올 컬럼

# 최종 선택할 컬럼 리스트 (matching_rows_df와 grouped_df에서 가져온 컬럼 모두 포함)
final_cols = final_cols_from_matching + pro_cols + jrcode_cols + final_cols_from_grouped

# 최종 데이터프레임 생성
final_df = merged_df[final_cols]
#final_df

# filtered_df에 있는 JRCODE 컬럼을 final_df에 병합
final_df = pd.merge(final_df, grouped_filtered_df.add_suffix('_c'), how='left', left_on='1_JRCODE', right_on='RUBBER_c')

### PHR 데이터 최대값 기준 전처리

In [11]:
%%time

# # PHR 그룹과 PHR_c 그룹의 컬럼 리스트
# phr_cols = [col for col in df.columns if 'PHR' in col and '_c' not in col]
# phr_c_cols = [col for col in df.columns if 'PHR' in col and '_c' in col]

# # 각 그룹에 대해 가장 큰 값이 100이 되도록 스케일링
# def scale_to_100(row, cols):
#     max_val = row[cols].max()
#     if max_val == 0:
#         return row
#     return row / max_val * 100

# 스케일링이 필요한 컬럼 그룹을 정의합니다.
phr_cols = [f'{i}_PHR' for i in range(1, 22)]
phr_c_cols = [f'{i}_PHR_c' for i in range(1, 22)]

# PHR 그룹 + PHR_C 그룹에 대한 스케일링
final_df[phr_cols+phr_c_cols] = final_df[phr_cols+phr_c_cols].apply(lambda x: x / x.max() * 100, axis=1)

CPU times: total: 44 s
Wall time: 44 s


### 컬럼명 별로 그룹 딕셔너리 생성

In [10]:
# 주어진 컬럼 리스트
columns = ['1_JRCODE', '2_JRCODE', '3_JRCODE', '4_JRCODE', '5_JRCODE', '6_JRCODE', '7_JRCODE', '8_JRCODE', '9_JRCODE', '10_JRCODE',
           '11_JRCODE', '12_JRCODE', '13_JRCODE', '14_JRCODE', '15_JRCODE', '16_JRCODE', '17_JRCODE', '18_JRCODE', '19_JRCODE', '20_JRCODE',
           '21_JRCODE', 'LOTNO', 'RUBBER', 'MECHCD', 'HS', 'HS_RESULT', 'SG', 'SG_RESULT', 'TS', 'TS_RESULT', 'EB', 'EB_RESULT', 'MNY',
           'MNY_RESULT', 'REHO_MIN', 'REHO_MAX', 'REHO_TS2', 'REHO_TC90', 'REHO_RESULT', 'SCR', 'SCR_RESULT', 'HS_TIME', 'HS_TEMP',
           'SG_TIME', 'SG_TEMP', 'TS_TIME', 'TS_TEMP', 'EB_TIME', 'EB_TEMP', 'MNY_TIME', 'MNY_TEMP', 'REHO_TIME', 'REHO_TEMP', 'SCR_TIME',
           'SCR_TEMP', 'RPM1', 'RPM2', 'RPM3', 'RPM4', 'RPM5', 'RPM6', 'RPM7', 'RPM8', 'RPM9', 'RPM10', 'RPM11', 'RPM12', 'RPM13',
           'RPM14', 'RPM15', 'RPM16', 'RPM17', 'RPM18', 'RPM19', 'RPM20', 'TEMP1', 'TEMP2', 'TEMP3', 'TEMP4', 'TEMP5', 'TEMP6',
           'TEMP7', 'TEMP8', 'TEMP9', 'TEMP10', 'TEMP11', 'TEMP12', 'TEMP13', 'TEMP14', 'TEMP15', 'TEMP16', 'TEMP17', 'TEMP18',
           'TEMP19', 'TEMP20', 'TIME1', 'TIME2', 'TIME3', 'TIME4', 'TIME5', 'TIME6', 'TIME7', 'TIME8', 'TIME9', 'TIME10', 'TIME11',
           'TIME12', 'TIME13', 'TIME14', 'TIME15', 'TIME16', 'TIME17', 'TIME18', 'TIME19', 'TIME20', 'JUK1', 'JUK2', 'JUK3',
           'JUK4', 'JUK5', 'JUK6', 'JUK7', 'JUK8', 'JUK9', 'JUK10', 'JUK11', 'JUK12', 'JUK13', 'JUK14', 'JUK15', 'JUK16',
           'JUK17', 'JUK18', 'JUK19', 'JUK20', '1_PHR', '2_PHR', '3_PHR', '4_PHR', '5_PHR', '6_PHR', '7_PHR', '8_PHR', '9_PHR',
           '10_PHR', '11_PHR', '12_PHR', '13_PHR', '14_PHR', '15_PHR', '16_PHR', '17_PHR', '18_PHR', '19_PHR', '20_PHR', '21_PHR',
           '1_PUTGB', '2_PUTGB', '3_PUTGB', '4_PUTGB', '5_PUTGB', '6_PUTGB', '7_PUTGB', '8_PUTGB', '9_PUTGB', '10_PUTGB', '11_PUTGB',
           '12_PUTGB', '13_PUTGB', '14_PUTGB', '15_PUTGB', '16_PUTGB', '17_PUTGB', '18_PUTGB', '19_PUTGB', '20_PUTGB', '21_PUTGB',
           'RPM1_c', 'RPM2_c', 'RPM3_c', 'RPM4_c', 'RPM5_c', 'RPM6_c', 'RPM7_c', 'RPM8_c', 'RPM9_c', 'RPM10_c', 'RPM11_c',
           'RPM12_c', 'RPM13_c', 'RPM14_c', 'RPM15_c', 'RPM16_c', 'RPM17_c', 'RPM18_c', 'RPM19_c', 'RPM20_c', 'TEMP1_c', 'TEMP2_c',
           'TEMP3_c', 'TEMP4_c', 'TEMP5_c', 'TEMP6_c', 'TEMP7_c', 'TEMP8_c', 'TEMP9_c', 'TEMP10_c', 'TEMP11_c', 'TEMP12_c',
           'TEMP13_c', 'TEMP14_c', 'TEMP15_c', 'TEMP16_c', 'TEMP17_c', 'TEMP18_c', 'TEMP19_c', 'TEMP20_c', 'TIME1_c', 'TIME2_c',
           'TIME3_c', 'TIME4_c', 'TIME5_c', 'TIME6_c', 'TIME7_c', 'TIME8_c', 'TIME9_c', 'TIME10_c', 'TIME11_c', 'TIME12_c',
           'TIME13_c', 'TIME14_c', 'TIME15_c', 'TIME16_c', 'TIME17_c', 'TIME18_c', 'TIME19_c', 'TIME20_c', 'JUK1_c', 'JUK2_c',
           'JUK3_c', 'JUK4_c', 'JUK5_c', 'JUK6_c', 'JUK7_c', 'JUK8_c', 'JUK9_c', 'JUK10_c', 'JUK11_c', 'JUK12_c', 'JUK13_c',
           'JUK14_c', 'JUK15_c', 'JUK16_c', 'JUK17_c', 'JUK18_c', 'JUK19_c', 'JUK20_c', '1_PHR_c', '2_PHR_c', '3_PHR_c',
           '4_PHR_c', '5_PHR_c', '6_PHR_c', '7_PHR_c', '8_PHR_c', '9_PHR_c', '10_PHR_c', '11_PHR_c', '12_PHR_c', '13_PHR_c',
           '14_PHR_c', '15_PHR_c', '16_PHR_c', '17_PHR_c', '18_PHR_c', '19_PHR_c', '20_PHR_c', '21_PHR_c', '1_PUTGB_c',
           '2_PUTGB_c', '3_PUTGB_c', '4_PUTGB_c', '5_PUTGB_c', '6_PUTGB_c', '7_PUTGB_c', '8_PUTGB_c', '9_PUTGB_c', '10_PUTGB_c',
           '11_PUTGB_c', '12_PUTGB_c', '13_PUTGB_c', '14_PUTGB_c', '15_PUTGB_c', '16_PUTGB_c', '17_PUTGB_c', '18_PUTGB_c', '19_PUTGB_c',
           '20_PUTGB_c', '21_PUTGB_c', 'RUBBER_c', '1_JRCODE_c', '2_JRCODE_c', '3_JRCODE_c', '4_JRCODE_c', '5_JRCODE_c',
           '6_JRCODE_c', '7_JRCODE_c', '8_JRCODE_c', '9_JRCODE_c', '10_JRCODE_c', '11_JRCODE_c', '12_JRCODE_c', '13_JRCODE_c',
           '14_JRCODE_c', '15_JRCODE_c', '16_JRCODE_c', '17_JRCODE_c', '18_JRCODE_c', '19_JRCODE_c', '20_JRCODE_c', '21_JRCODE_c']

# 각 그룹에 해당하는 컬럼명을 포함한 리스트 생성
groups = ['RPM', 'TEMP', 'TIME', 'JUK', 'PUTGB', 'PHR', 'JRCODE']

# 컬럼명에 따라 그룹화
grouped_columns = {}
for group in groups:
    grouped_columns[group] = [col for col in columns if group in col]

# # 그룹화된 컬럼 출력
# for group, cols in grouped_columns.items():
#     print(f'{group} 그룹: {cols}')
    
# 제거할 값들
remove_values = ['HS_TEMP', 'SG_TEMP', 'TS_TEMP', 'HS_TIME', 'SG_TIME', 'TS_TIME', 'EB_TIME',  'MNY_TIME','REHO_TIME','SCR_TIME',
                'EB_TEMP','MNY_TEMP','REHO_TEMP','SCR_TEMP']

# 딕셔너리에서 특정 값들만 제거
for key in grouped_columns.keys():
    grouped_columns[key] = [value for value in grouped_columns[key] if value not in remove_values]

# 새로운 딕셔너리를 생성합니다.
new_group_dict = {}

# 각 그룹별로 작업을 수행합니다.
for group_name, group_cols in grouped_columns.items():
    # _c가 붙지 않은 컬럼들
    non_c_cols = [col for col in group_cols if not col.endswith('_c')]
    
    # _c가 붙은 컬럼들
    c_cols = [col for col in group_cols if col.endswith('_c')]
    
    # 새로운 그룹 이름을 만들고 컬럼을 할당합니다.
    new_group_dict[f'{group_name}_non_c'] = non_c_cols
    new_group_dict[f'{group_name}_c'] = c_cols
    
new_group_dict_ori = new_group_dict    
new_group_dict['JRCODE_non_c'].remove('1_JRCODE')
new_group_dict['PHR_non_c'].remove('1_PHR')
new_group_dict['PUTGB_non_c'].remove('1_PUTGB')
print('완료')

완료


### 각 접두사별로 분리되어 있던 컬럼을 합쳐서 리스트로 만듬

In [11]:
# 그룹을 딕셔너리로 정의합니다.
# 이 부분에서는 grouped_columns 딕셔너리를 사용합니다. (이전에 생성한 것을 사용)
group_dict = new_group_dict

# 빈 데이터프레임을 생성합니다.
combined_df = pd.DataFrame()

# LOTNO 컬럼을 설정합니다.
combined_df['LOTNO'] = final_df['LOTNO']

# 각 그룹별로 작업을 수행합니다.
for group_name, group_cols in group_dict.items():
    # 그룹에 해당하는 컬럼만 선택하여 임시 데이터프레임을 만듭니다.
    tmp_df = final_df[group_cols]
    
    # NaN을 제외하고 리스트로 합치기
    combined_df[f'{group_name}_ls'] = tmp_df.apply(lambda row: [x for x in row if pd.notna(x)], axis=1)
    
    # 리스트의 길이를 새로운 컬럼에 저장 (None 또는 NaN을 제외)
    combined_df[f'{group_name}_num'] = combined_df[f'{group_name}_ls'].apply(lambda x: len(x))

## 리스트 데이터 전처리

### 맨 뒤에 연속되는 0값(Nan으로 간주)을 제거

In [12]:
# 연속된 0을 제거하는 함수
def remove_trailing_zeros(lst):
    while lst and lst[-1] == 0:
        lst.pop()
    return lst

# 각 그룹별로 작업을 수행합니다.
for group_name in group_dict.keys():
    # 연속된 0 제거
    combined_df[f'{group_name}_ls'] = combined_df[f'{group_name}_ls'].apply(remove_trailing_zeros)
    
    # 리스트의 길이를 다시 새로운 컬럼에 저장 (None 또는 NaN을 제외)
    combined_df[f'{group_name}_num'] = combined_df[f'{group_name}_ls'].apply(lambda x: len(x))

### _c그룹과 _c가 붙지 않은 접미사 그룹 리스트끼리 Merge

In [13]:
# 컬럼 이름에서 고유한 그룹 이름을 추출합니다.
unique_groups = set(col.split('_')[0] for col in combined_df.columns if '_ls' in col)

# 각 고유한 그룹에 대해 작업을 수행합니다.
for group in unique_groups:
    # 해당 그룹에 속하는 '_ls'가 포함된 컬럼을 찾습니다.
    # 이번에는 '_non_c'가 뒤로 오도록 정렬합니다.
    ls_cols = sorted([col for col in combined_df.columns if group in col and '_ls' in col], key=lambda x: '_non_c' in x)
    
    # 리스트를 합칩니다.
    combined_df[f'{group}_combined_ls'] = combined_df[ls_cols].apply(lambda row: [x for sublist in row for x in sublist], axis=1)

    # 리스트의 길이를 새로운 컬럼에 저장합니다. (None 또는 NaN을 제외)
    combined_df[f'{group}_combined_num'] = combined_df[f'{group}_combined_ls'].apply(lambda x: len(x) if x is not None else 0)

### 전처리한 리스트 데이터 중에 최종적으로 FMB데이터에 Merge 시킬 데이터 정의

In [14]:
filter_df = combined_df[['TEMP_combined_ls', 'PHR_combined_ls', 'PUTGB_combined_ls', 'RPM_combined_ls', 'TIME_combined_ls',
                         'JRCODE_combined_ls', 'JUK_combined_ls']]

### 원본데이터에서 리스트로 묶어서 들어가야하는 컬럼들 삭제

In [15]:
# 딕셔너리의 모든 값들을 하나의 리스트로 풀어내기 (flatten) in one-liner

# extend() 메서드를 사용하여 하나의 리스트에 다른 리스트의 원소를 추가
flattened_list_one_liner = [item for sublist in new_group_dict_ori.values() for item in sublist]

# rpm, rpm_c 등을 제거
final_df_filter = final_df.drop(flattened_list_one_liner, axis=1)

### []가 있는 데이터를 실제 리스트 형식의 데이터로 변경

In [16]:
import ast
# 문자열이 리스트 형태인지 확인하는 함수 정의
def is_list_like(s):
    try:
        ast.literal_eval(s)
        return True
    except (ValueError, SyntaxError):
        return False

# 문자열을 리스트로 변환하는 함수 정의
def convert_to_list(s):
    if is_list_like(s):
        return ast.literal_eval(s)
    return s

# 문자열을 리스트로 변환
for col in filter_df.columns:
    filter_df[col] = filter_df[col].apply(convert_to_list)

### C1, F1합쳐진 리스트 패딩작업

In [17]:
# 특정 컬럼에 대해서만 가장 긴 리스트의 길이를 찾는다.
specified_cols = ['TEMP_combined_ls','RPM_combined_ls','JUK_combined_ls','TIME_combined_ls']
tmp_df = filter_df[specified_cols]
tmp_df['max_len'] = tmp_df.applymap(lambda x: len(x) if isinstance(x, list) else 0).max(axis=1)

def pad_list(lst, target_length):
    # 리스트의 길이가 target_length보다 작은 경우에만 패딩을 적용
    if len(lst) < target_length:
        return lst + [0.0] * (target_length - len(lst))
    return lst

# 각 행에 대해 패딩을 적용
for idx, row in tmp_df.iterrows():
    max_len = row['max_len']
    for col in ['TEMP_combined_ls', 'RPM_combined_ls', 'JUK_combined_ls', 'TIME_combined_ls']:
        tmp_df.at[idx, col] = pad_list(row[col], max_len)

# 결과 변경
filter_df[specified_cols] = tmp_df[specified_cols]

# 이제 각 컬럼별로 가장 긴 리스트의 길이를 찾는다.
max_len_dict = filter_df.applymap(lambda x: len(x) if isinstance(x, list) else 0).max()

# 각 컬럼별로 패딩을 적용한다.
filter_df_padded = pd.DataFrame()
for col in filter_df.columns:
    max_len = max_len_dict[col]
    padded_col = filter_df[col].apply(lambda x: x + [np.nan] * (max_len - len(x)) if isinstance(x, list) else x)
    filter_df_padded[col] = padded_col

# 리스트의 원소를 별도의 컬럼으로 분리한다.
filter_df_expanded = pd.DataFrame()
for col in filter_df_padded.columns:
    if isinstance(filter_df_padded[col][0], list):
        max_len = max_len_dict[col]
        expanded = pd.DataFrame(filter_df_padded[col].to_list(), columns=[f"{col}_{i+1}" for i in range(max_len)])
        filter_df_expanded = pd.concat([filter_df_expanded, expanded], axis=1)
    else:
        filter_df_expanded[col] = filter_df_padded[col]

### combined_ls같은 컬럼의 이름을 원래 데이터셋이 가지고 있었던 컬럼 이름으로 통일해줌

In [18]:
# 사용자가 지정한 컬럼 이름 딕셔너리
col_names_dict_1 = {
    'RPM_combined_ls': ['RPM1','RPM2','RPM3','RPM4','RPM5','RPM6','RPM7','RPM8','RPM9','RPM10','RPM11','RPM12','RPM13','RPM14','RPM15','RPM16','RPM17','RPM18','RPM19','RPM20'],
    'TEMP_combined_ls': ['TEMP1','TEMP2','TEMP3','TEMP4','TEMP5','TEMP6','TEMP7','TEMP8','TEMP9','TEMP10','TEMP11','TEMP12','TEMP13','TEMP14','TEMP15','TEMP16','TEMP17','TEMP18','TEMP19','TEMP20'],
    'TIME_combined_ls': ['TIME1','TIME2','TIME3','TIME4','TIME5','TIME6','TIME7','TIME8','TIME9','TIME10','TIME11','TIME12','TIME13','TIME14','TIME15','TIME16','TIME17','TIME18','TIME19','TIME20'],
    'JUK_combined_ls': ['JUK1','JUK2','JUK3','JUK4','JUK5','JUK6','JUK7','JUK8','JUK9','JUK10','JUK11','JUK12','JUK13','JUK14','JUK15','JUK16','JUK17','JUK18','JUK19','JUK20'],
}
col_names_dict_2 = {
    'JRCODE_combined_ls': ['1_JRCODE','2_JRCODE','3_JRCODE','4_JRCODE','5_JRCODE','6_JRCODE','7_JRCODE','8_JRCODE','9_JRCODE','10_JRCODE','11_JRCODE','12_JRCODE','13_JRCODE','14_JRCODE','15_JRCODE','16_JRCODE','17_JRCODE','18_JRCODE','19_JRCODE','20_JRCODE','21_JRCODE'],
    'PHR_combined_ls': ['1_PHR','2_PHR','3_PHR','4_PHR','5_PHR','6_PHR','7_PHR','8_PHR','9_PHR','10_PHR','11_PHR','12_PHR','13_PHR','14_PHR','15_PHR','16_PHR','17_PHR','18_PHR','19_PHR','20_PHR','21_PHR'],
    'PUTGB_combined_ls': ['1_PUTGB','2_PUTGB','3_PUTGB','4_PUTGB','5_PUTGB','6_PUTGB','7_PUTGB','8_PUTGB','9_PUTGB','10_PUTGB','11_PUTGB','12_PUTGB','13_PUTGB','14_PUTGB','15_PUTGB','16_PUTGB','17_PUTGB','18_PUTGB','19_PUTGB','20_PUTGB','21_PUTGB']
}

# 각 prefix에 대해 컬럼 이름을 동적으로 변경합니다.
for prefix, new_cols in col_names_dict_1.items():
    # 데이터프레임에서 해당 prefix로 시작하는 컬럼들을 찾습니다.
    old_cols = [col for col in filter_df_expanded.columns if col.startswith(prefix)]
    
    # 원본 데이터프레임에 있는 컬럼의 개수에 맞춰 새로운 컬럼 이름을 생성합니다.
    if len(old_cols) > len(new_cols):
        # 기존에 지정한 이름에 번호를 추가하여 새로운 컬럼 이름을 생성합니다.
        base_name = new_cols[0].rstrip('1234567890')  # 숫자를 제거한 기본 이름을 얻습니다.
        extra_cols = [f"{base_name}{i}" for i in range(len(new_cols) + 1, len(old_cols) + 1)]
        new_cols.extend(extra_cols)
    
    # 컬럼 이름을 변경합니다.
    rename_dict = {old_cols[i]: new_cols[i] for i in range(len(old_cols))}
    filter_df_expanded.rename(columns=rename_dict, inplace=True)
    
# 각 prefix에 대해 컬럼 이름을 동적으로 변경합니다.
for prefix, new_cols in col_names_dict_2.items():
    # 데이터프레임에서 해당 prefix로 시작하는 컬럼들을 찾습니다.
    old_cols = [col for col in filter_df_expanded.columns if col.startswith(prefix)]
    
    # 원본 데이터프레임에 있는 컬럼의 개수에 맞춰 새로운 컬럼 이름을 생성합니다.
    if len(old_cols) > len(new_cols):
        # 기존에 지정한 이름에 번호를 추가하여 새로운 컬럼 이름을 생성합니다.
        base_name = new_cols[0].rstrip('1234567890')  # 숫자를 제거한 기본 이름을 얻습니다.
        extra_cols = [f"{i}_{base_name.split('_')[1]}" for i in range(len(new_cols) + 1, len(old_cols) + 1)]
        new_cols.extend(extra_cols)
    
    # 컬럼 이름을 변경합니다.
    rename_dict = {old_cols[i]: new_cols[i] for i in range(len(old_cols))}
    filter_df_expanded.rename(columns=rename_dict, inplace=True)
    
# 필요없거나 중복된 컬럼 제거
temp = final_df_filter.drop(['RUBBER_c','1_JRCODE','1_PHR','1_PUTGB'], axis=1)

### 처리된 X(공정조건, 재료코드 리스트)데이터와 Y(물성, 불량여부) 데이터를 Merge

In [19]:
filter_df_expanded_pre = pd.concat([temp, filter_df_expanded], axis=1)

## 완성된 데이터셋 전처리

### RESULT라는 단어가 들어간 컬럼에 대해 P,F 전처리

In [20]:
# 'RESULT' 문자열이 들어가는 컬럼만 선택
result_columns = [col for col in filter_df_expanded_pre.columns if 'RESULT' in col]

# P는 1로, F는 0으로 변환
for col in result_columns:
    filter_df_expanded_pre[col] = filter_df_expanded_pre[col].map({'P': 1, 'F': 0})

### TIME이라는 변수가 들어간 컬럼에 대해 이상치(연산자) 전처리

In [21]:
# TIME 컬럼 안에 이상치들이 있어서 처리

import re
# 'TIME' 문자열이 들어가는 컬럼만 선택
time_columns = [col for col in filter_df_expanded_pre.columns if 'TIME' in col]

# 연산자 적용 함수
def apply_operator(expression):
    if isinstance(expression, (int, float)):
        return expression
    
    if re.match("^[0-9+\-*/]+$", expression):
        try:
            return eval(expression)
        except ZeroDivisionError:
            return "Division by zero"
        except Exception as e:
            return str(e)
    else:
        return expression

# 각 'TIME' 컬럼에 연산자 적용
for col in time_columns:
    filter_df_expanded_pre[col] = filter_df_expanded_pre[col].apply(apply_operator)

## 분류, 회귀 데이터셋 구축

### 0삭제 데이터셋

In [23]:
# 0삭제 데이터셋
basic_cols = ['LOTNO','RUBBER']
# 변수명 조정 필요
# ========================================================
rpm_cols = ['RPM1','RPM2','RPM3','RPM4','RPM5','RPM6','RPM7','RPM8','RPM9','RPM10','RPM11','RPM12','RPM13','RPM14','RPM15','RPM16','RPM17','RPM18','RPM19','RPM20','RPM21','RPM22','RPM23']
temp_cols = ['TEMP1','TEMP2','TEMP3','TEMP4','TEMP5','TEMP6','TEMP7','TEMP8','TEMP9','TEMP10','TEMP11','TEMP12','TEMP13','TEMP14','TEMP15','TEMP16','TEMP17','TEMP18','TEMP19','TEMP20','TEMP21','TEMP22','TEMP23']
time_cols = ['TIME1','TIME2','TIME3','TIME4','TIME5','TIME6','TIME7','TIME8','TIME9','TIME10','TIME11','TIME12','TIME13','TIME14','TIME15','TIME16','TIME17','TIME18','TIME19','TIME20','TIME21','TIME22','TIME23']
juk_cols = ['JUK1','JUK2','JUK3','JUK4','JUK5','JUK6','JUK7','JUK8','JUK9','JUK10','JUK11','JUK12','JUK13','JUK14','JUK15','JUK16','JUK17','JUK18','JUK19','JUK20','JUK21','JUK22','JUK23']
jrcode_cols = ['1_JRCODE','2_JRCODE','3_JRCODE','4_JRCODE','5_JRCODE','6_JRCODE','7_JRCODE','8_JRCODE','9_JRCODE','10_JRCODE','11_JRCODE','12_JRCODE','13_JRCODE','14_JRCODE','15_JRCODE','16_JRCODE','17_JRCODE','18_JRCODE','19_JRCODE','20_JRCODE','21_JRCODE','22_JRCODE','23_JRCODE','24_JRCODE','25_JRCODE']
phr_cols = ['1_PHR','2_PHR','3_PHR','4_PHR','5_PHR','6_PHR','7_PHR','8_PHR','9_PHR','10_PHR','11_PHR','12_PHR','13_PHR','14_PHR','15_PHR','16_PHR','17_PHR','18_PHR','19_PHR','20_PHR','21_PHR','22_PHR','23_PHR','24_PHR','25_PHR']
putgb_cols = ['1_PUTGB','2_PUTGB','3_PUTGB','4_PUTGB','5_PUTGB','6_PUTGB','7_PUTGB','8_PUTGB','9_PUTGB','10_PUTGB','11_PUTGB','12_PUTGB','13_PUTGB','14_PUTGB','15_PUTGB','16_PUTGB','17_PUTGB','18_PUTGB','19_PUTGB','20_PUTGB','21_PUTGB','22_PUTGB','23_PUTGB','24_PUTGB','25_PUTGB']
# 변수명 조정 필요
# ========================================================

trg_cols = ['HS','SG','TS','EB']
trg_yCols = ['HS_VAL','SG_VAL','TS_VAL','EB_VAL']
trg_ispt_cols = ['HS_TEMP','HS_TIME','SG_TEMP','SG_TIME','TS_TEMP','TS_TIME','EB_TEMP','EB_TIME']

data_df = filter_df_expanded_pre.copy()

fin_df = data_df[basic_cols+rpm_cols+temp_cols+time_cols+juk_cols+jrcode_cols+phr_cols+putgb_cols+trg_ispt_cols+trg_cols]
fin_df.columns = basic_cols+rpm_cols+temp_cols+time_cols+juk_cols+jrcode_cols+phr_cols+putgb_cols+trg_ispt_cols+trg_yCols

for yCol in trg_yCols:
    fin_df = fin_df[~(fin_df[yCol].isna())&(fin_df[yCol]!=0)]

fin_df.to_csv(f'./tmp_dataset/mes_fmb_pps_whole_data_fin_noZero.csv', index=False)
display(fin_df.shape)
fin_df

(17478, 181)

,LOTNO,RUBBER,RPM1,RPM2,RPM3,RPM4,RPM5,RPM6,RPM7,RPM8,RPM9,RPM10,RPM11,RPM12,RPM13,RPM14,RPM15,RPM16,RPM17,RPM18,RPM19,RPM20,RPM21,RPM22,RPM23,TEMP1,TEMP2,TEMP3,TEMP4,TEMP5,TEMP6,TEMP7,TEMP8,TEMP9,TEMP10,TEMP11,TEMP12,TEMP13,TEMP14,TEMP15,TEMP16,TEMP17,TEMP18,TEMP19,TEMP20,TEMP21,TEMP22,TEMP23,TIME1,TIME2,TIME3,TIME4,TIME5,TIME6,TIME7,TIME8,TIME9,TIME10,TIME11,TIME12,TIME13,TIME14,TIME15,TIME16,TIME17,TIME18,TIME19,TIME20,TIME21,TIME22,TIME23,JUK1,JUK2,JUK3,JUK4,JUK5,JUK6,JUK7,JUK8,JUK9,JUK10,JUK11,JUK12,JUK13,JUK14,JUK15,JUK16,JUK17,JUK18,JUK19,JUK20,JUK21,JUK22,JUK23,1_JRCODE,2_JRCODE,3_JRCODE,4_JRCODE,5_JRCODE,6_JRCODE,7_JRCODE,8_JRCODE,9_JRCODE,10_JRCODE,11_JRCODE,12_JRCODE,13_JRCODE,14_JRCODE,15_JRCODE,16_JRCODE,17_JRCODE,18_JRCODE,19_JRCODE,20_JRCODE,21_JRCODE,22_JRCODE,23_JRCODE,24_JRCODE,25_JRCODE,1_PHR,2_PHR,3_PHR,4_PHR,5_PHR,6_PHR,7_PHR,8_PHR,9_PHR,10_PHR,11_PHR,12_PHR,13_PHR,14_PHR,15_PHR,16_PHR,17_PHR,18_PHR,19_PHR,20_PHR,21_PHR,22_PHR,23_PHR,24_PHR,25_PHR,1_PUTGB,2_PUTGB,3_PUTGB,4_PUTGB,5_PUTGB,6_PUTGB,7_PUTGB,8_PUTGB,9_PUTGB,10_PUTGB,11_PUTGB,12_PUTGB,13_PUTGB,14_PUTGB,15_PUTGB,16_PUTGB,17_PUTGB,18_PUTGB,19_PUTGB,20_PUTGB,21_PUTGB,22_PUTGB,23_PUTGB,24_PUTGB,25_PUTGB,HS_TEMP,HS_TIME,SG_TEMP,SG_TIME,TS_TEMP,TS_TIME,EB_TEMP,EB_TIME,HS_VAL,SG_VAL,TS_VAL,EB_VAL
1449,G0120I230005001,2630-A1.00-F1,60.0,60.0,60.0,60.0,41.0,41.0,36.0,36.0,20.000000,21.0,21.000000,21.0,21.0,21.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.242857,80.642857,83.100000,72.657143,110.457143,115.128571,168.828571,150.442857,59.100000,110.700000,107.600000,107.900000,107.50,118.5,104.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.442857,9.800000,44.742857,31.342857,36.114286,8.142857,68.442857,16.800000,247.100000,51.900000,8.500000,23.400000,8.4,31.9,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.571429,29.428571,133.571429,58.428571,80.857143,8.571429,176.000000,60.714286,237.000000,133.000000,5.000000,27.000000,14.000000,69.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ER001,ES002,CB001,CC006,CB003,NN220,OA001,CA007,CA011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.214286,18.600000,2.610000,1.310,5.200,59.677143,3.260,0.787,0.131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,2.0,6.0,6.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,10.0,160.0,10.0,160.0,10.0,160.0,10.0,62.0,1.106,284.0,507.0
1507,G0119K070010001,267T-A5.01-F1,41.0,41.0,41.0,41.0,41.0,36.0,36.0,36.0,36.000000,35.0,36.000000,36.0,36.0,20.0,21.0,21.0,20.0,21.0,21.0,21.0,NaN,NaN,NaN,130.718421,88.794737,90.505263,91.471053,86.242105,122.726316,123.636842,132.642105,136.223684,139.234211,142.726316,176.057895,161.15,57.2,96.5,96.9,105.1,104.0,114.3,110.2,NaN,NaN,NaN,12.444737,9.700000,54.239474,7.821053,10.276316,34.300000,17.484211,19.300000,8.084211,14.268421,8.197368,37.926316,16.8,189.3,42.0,7.7,39.3,7.8,35.6,16.9,NaN,NaN,NaN,30.342105,16.947368,91.973684,5.631579,14.052632,66.868421,42.973684,23.657895,17.421053,21.947368,14.210526,82.815789,63.315789,264.0,88.0,15.0,52.0,7.0,113.0,27.0,NaN,NaN,NaN,ER001,EB001,CJ001,CB001,CC001,CC002,CW001,CB003,NN234,NN234,OA001,CV001,CA007,CE001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.773684,47.805263,0.234211,1.790,1.790,1.790000,1.790,4.700,48.300,22.073684,9.570,3.040000,0.852000,0.304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,6.0,6.0,6.0,6.0,2.0,3.0,3.0,6.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,10.0,160.0,10.0,160.0,10.0,160.0,10.0,65.0,1.127,221.0,527.0
1525,G0120A280022001,267T-A5.01-F1,41.0,41.0,41.0,41.0,41.0,36.0,36.0,36.0,36.000000,35.0,36.000000,36.0,36.0,19.0,21.0,21.0,21.0,20.0,21.0,21.0,NaN,NaN,NaN,130.718421,88.794737,90.505263,91.471053,86.242105,122.726316,123.636842,132.642105,136.223684,139.234211,142.726316,176.057895,161.15,57.2,110.5,105.3,113.6,113.7,121.6,116.0,NaN,NaN,NaN,12.444737,9.700000,54.239474,7.821053,10.276316,34.300000,17.484211,19.300000,8.084211,14.268421,8.197368,37.926316,16.8,45.9,51.3,7.7,39.2,

In [4]:
fin_df = pd.read_csv(f'./tmp_dataset/mes_fmb_pps_whole_data_fin_noZero.csv')
tmp_df = fin_df.copy()

# 원재료 코드 리스트 생성
JRCODE_ls = []
jrcode_col_ls = [col for col in tmp_df.columns if 'JRCODE' in col]
for jrcode_col in jrcode_col_ls:
    JRCODE_ls += tmp_df[jrcode_col].unique().tolist()
    JRCODE_ls = list(set(JRCODE_ls))
    JRCODE_ls = [x for x in JRCODE_ls if str(x) != 'nan']    

[jr for jr in JRCODE_ls if 'C1' in jr]

['240C-A1.01-C1', '4450-Y1.00-C1']